In [17]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2021-04-08 03:16:56--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2021-04-08 03:16:56 (217 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [18]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(), # Flatten the results to feed into a DNN
  tf.keras.layers.Dense(512, activation='relu'), # 512 neuron hidden layer
  tf.keras.layers.Dense(1, activation='sigmoid'),
])


from tensorflow.keras.optimizers import RMSprop

model.compile(loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [13]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(300, 300),
        batch_size=8,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [19]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      steps_per_epoch=10,
      epochs=20,
      callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
10/10 [==============================] - 11s 1s/step - loss: 4.0546 - accuracy: 0.5853
Epoch 2/20
10/10 [==============================] - 9s 936ms/step - loss: 0.8477 - accuracy: 0.6622
Epoch 3/20
10/10 [==============================] - 10s 1s/step - loss: 0.2766 - accuracy: 0.9474
Epoch 4/20
10/10 [==============================] - 10s 1s/step - loss: 0.1236 - accuracy: 0.9822
Epoch 5/20
10/10 [==============================] - 9s 921ms/step - loss: 0.0683 - accuracy: 0.9704
Epoch 6/20
10/10 [==============================] - 10s 977ms/step - loss: 0.0221 - accuracy: 1.0000

Reached 99.9% accuracy so cancelling training!
